In [1]:
import pandas as pd
import repo.repo as repo
import repo.memory_handler as memory_handler
from repo.repo_objects import RepoInfoKey, MeasureConfiguration
from job_runner.job_runner import SimpleJobRunner, JobState
import logging as logging

#FORMAT = "[%(filename)s:%(lineno)s - %(funcName)20s() ] %(message)s"
#logging.basicConfig(format=FORMAT, level=logging.DEBUG)
logging.basicConfig(level=logging.ERROR)

# Read housing data

In [2]:
data = pd.read_csv('housing.csv')

# Create a new repository

In [3]:
# setting up the repository
handler = memory_handler.RepoObjectMemoryStorage()
numpy_handler = memory_handler.NumpyMemoryStorage()
job_runner = SimpleJobRunner(None)
ml_repo = repo.MLRepo('test_user', handler, numpy_handler, handler, job_runner)
job_runner.set_repo(ml_repo)
ml_repo._job_runner = job_runner

## Adding data
The data in the repository is handled by two different data objects:
- RawData is the object containing real data.
- DataSet is the object conaining the logical data, i.e. a reference to a RawData object together with a specification, which data from the RawData will be used

Normally one will add RawData and then define DataSets which are used to train or test a model which is exactly the way shown in the following.

In [4]:
# Add RawData. A convenient way to add RawData is simply to use the method add_data.
# This method just takes a pandas dataframe and the specification, which columns belong to the input 
#and which to the targets.
ml_repo.add_data('boston_housing', data, input_variables=['RM', 'LSTAT', 'PTRATIO'], target_variables = ['MEDV'])

In [5]:
# create DataSet objects for training and test data
training_data = repo.DataSet('boston_housing', 0, 300, 
                            repo_info = {RepoInfoKey.NAME.value: 'training_data', RepoInfoKey.CATEGORY.value: repo.MLObjectType.TRAINING_DATA})
test_data = repo.DataSet('boston_housing', 301, -1, 
                            repo_info = {RepoInfoKey.NAME.value: 'test_data',  RepoInfoKey.CATEGORY.value: repo.MLObjectType.TEST_DATA})
# add the objects to the repository
ml_repo.add([training_data, test_data], message = 'add training and test data')

{'repo_mapping': 1, 'test_data': 0, 'training_data': 0}

## Adding sklearn model
In the following we want to use models from the sklearn package. For the sklearn package, there is a simple module (externals.sklearn_interface) interfacing 
the sklearn package so that this can be used within the repository. This interface prvides a simple method (add_model) to add an arbitrary sklearn model as a model which can be handled by the repository. This method adds a bunch of repo objects to the repository:
- An object defining the function to be called to evaluate the model
- An object defining the function to be called to train the model
- An object defining the model
- An object defining the model parameter

In [6]:
import externals.sklearn_interface as sklearn_interface
from sklearn.tree import DecisionTreeRegressor
sklearn_interface.add_model(ml_repo, DecisionTreeRegressor(), model_param={'max_depth': 5})

## Train the model

In [7]:
job_id = ml_repo.run_training()  

## Run evaluation
To measure errors and to provide plots the model must be evaluated on all test and training datasets.

In [8]:
job_id = ml_repo.run_evaluation()

## Add and compute measures

In [9]:
ml_repo.add_measure(MeasureConfiguration.MAX)
ml_repo.add_measure(MeasureConfiguration.R2)

In [10]:
job_ids = ml_repo.run_measures()

In [11]:
max_measure = ml_repo._get('DecisionTreeRegressor/measure/training_data/max')
print(str(max_measure.value))
max_measure = ml_repo._get('DecisionTreeRegressor/measure/test_data/max')
print(str(max_measure.value))


223200.0
415450.0


## Working with the repository

In [12]:
for k in repo.MLObjectType:
    names = ml_repo.get_names(k.value)
    for n in names: 
        print(n + '\t  ' + k.value)

DecisionTreeRegressor/eval/test_data	  eval_data
DecisionTreeRegressor/eval/training_data	  eval_data
boston_housing	  raw_data
training_data	  training_data
test_data	  test_data
DecisionTreeRegressor/model_param	  model_param
train_sklearn	  training_function
eval_sklearn	  model_eval_function
DecisionTreeRegressor	  model
DecisionTreeRegressor/model	  calib_model
CommitInfo	  commit_info
repo_mapping	  mapping
DecisionTreeRegressor/measure/test_data/max	  measure
DecisionTreeRegressor/measure/test_data/r2	  measure
DecisionTreeRegressor/measure/training_data/max	  measure
DecisionTreeRegressor/measure/training_data/r2	  measure
measure_config	  measure_config


In [13]:
for k in ml_repo.get_commits():
    print(str(k))

time: 2018-09-29 07:00:52.832833, author: test_user, message: data boston_housing added to repository, objects: {'boston_housing': 0, 'repo_mapping': 0}
time: 2018-09-29 07:00:52.885692, author: test_user, message: add training and test data, objects: {'training_data': 0, 'test_data': 0, 'repo_mapping': 1}
time: 2018-09-29 07:00:53.625163, author: test_user, message: add model evaluation function eval_sklearn, objects: {'eval_sklearn': 0, 'repo_mapping': 2}
time: 2018-09-29 07:00:53.625163, author: test_user, message: add model training function train_sklearn, objects: {'train_sklearn': 0, 'repo_mapping': 3}
time: 2018-09-29 07:00:53.625163, author: test_user, message: adding model and training parameter, objects: {'DecisionTreeRegressor/model_param': 0, 'repo_mapping': 4}
time: 2018-09-29 07:00:53.625163, author: test_user, message: add model DecisionTreeRegressor, objects: {'DecisionTreeRegressor': 0, 'repo_mapping': 5}
time: 2018-09-29 07:00:53.640855, author: test_user, message: tr

In [14]:
for k, v in job_runner._job_info.items():
    print(str(k) + ':  ' + str(v))
#job_runner.get_info('34484a2c-c225-11e8-9693-fc084a6691eb')

a47b79e6-c3a4-11e8-9db9-fc084a6691eb:  test_user, successfully_finished, started 2018-09-29 07:00:53.640855, finished 2018-09-29 07:00:53.640855
a4849bc6-c3a4-11e8-bc80-fc084a6691eb:  test_user, successfully_finished, started 2018-09-29 07:00:53.700704, finished 2018-09-29 07:00:53.702701
a484e9c6-c3a4-11e8-89dd-fc084a6691eb:  test_user, successfully_finished, started 2018-09-29 07:00:53.702701, finished 2018-09-29 07:00:53.703698
a49442f4-c3a4-11e8-ad18-fc084a6691eb:  test_user, successfully_finished, started 2018-09-29 07:00:53.803288, finished 2018-09-29 07:00:53.804285
a49469e8-c3a4-11e8-be53-fc084a6691eb:  test_user, successfully_finished, started 2018-09-29 07:00:53.804285, finished 2018-09-29 07:00:53.806279
a494b7cc-c3a4-11e8-935f-fc084a6691eb:  test_user, successfully_finished, started 2018-09-29 07:00:53.806279, finished 2018-09-29 07:00:53.807277
a494dec8-c3a4-11e8-a3b2-fc084a6691eb:  test_user, successfully_finished, started 2018-09-29 07:00:53.807277, finished 2018-09-29 0

## Change model parameter, check consistency and train

In [15]:
param = ml_repo._get('DecisionTreeRegressor/model_param')
param.sklearn_params['max_depth'] = 2
version = ml_repo.add(param)

In [16]:
import repo.tools as tools
#depp = ml_repo._get('DecisionTreeRegressor/model_param')
results = tools.check_model(ml_repo, 'DecisionTreeRegressor')
print(results)

{'DecisionTreeRegressor/model_param': {'modifier version': 0, 'latest version': 1}}


In [17]:
ml_repo.run_training()

UUID('a4c9d806-c3a4-11e8-8e5d-fc084a6691eb')

In [18]:
results = tools.check_model(ml_repo, 'DecisionTreeRegressor')
print(results)

{}


In [19]:
ml_repo.run_evaluation()
ml_repo.run_measures()

[UUID('a4da477a-c3a4-11e8-9737-fc084a6691eb'),
 UUID('a4da6e6e-c3a4-11e8-ac4f-fc084a6691eb'),
 UUID('a4da9564-c3a4-11e8-9819-fc084a6691eb'),
 UUID('a4dabc5a-c3a4-11e8-9e4f-fc084a6691eb')]

In [20]:
measure = ml_repo._get('DecisionTreeRegressor/measure/test_data/r2',version = (0,100))
for x in measure:
    print(str(x.repo_info))

{'version': 0, 'name': 'DecisionTreeRegressor/measure/test_data/r2', 'history': None, 'classname': 'repo.repo_objects.Measure', 'modification_info': {'DecisionTreeRegressor/eval/test_data': 0, 'DecisionTreeRegressor/model': 0, 'DecisionTreeRegressor/model_param': 0, 'training_data': 0, 'DecisionTreeRegressor': 0, 'test_data': 0}, 'description': None, 'category': 'measure', 'big_objects': [], 'commit_message': 'computing  measure r2 on data test_data'}
{'version': 1, 'name': 'DecisionTreeRegressor/measure/test_data/r2', 'history': None, 'classname': 'repo.repo_objects.Measure', 'modification_info': {'DecisionTreeRegressor/eval/test_data': 1, 'DecisionTreeRegressor/model': 1, 'DecisionTreeRegressor/model_param': 1, 'training_data': 0, 'DecisionTreeRegressor': 0, 'test_data': 0}, 'description': None, 'category': 'measure', 'big_objects': [], 'commit_message': 'computing  measure r2 on data test_data'}


In [21]:
ml_repo.get_names(repo.MLObjectType.MEASURE_CONFIGURATION)

['measure_config']

In [22]:
m = ml_repo._get('DecisionTreeRegressor/measure/training_data/r2')

In [23]:
str(m.repo_info)

"{'version': 1, 'name': 'DecisionTreeRegressor/measure/training_data/r2', 'history': None, 'classname': 'repo.repo_objects.Measure', 'modification_info': {'DecisionTreeRegressor/eval/training_data': 1, 'DecisionTreeRegressor/model': 1, 'DecisionTreeRegressor/model_param': 1, 'training_data': 0, 'DecisionTreeRegressor': 0}, 'description': None, 'category': 'measure', 'big_objects': [], 'commit_message': 'computing  measure r2 on data training_data'}"

In [24]:
print(results)

{}


In [25]:
data = ml_repo._get('boston_housing')

In [26]:
print(str(data.repo_info))

{'version': 0, 'name': 'boston_housing', 'history': None, 'classname': 'repo.repo_objects.RawData', 'modification_info': {}, 'description': None, 'category': <MLObjectType.RAW_DATA: 'raw_data'>, 'big_objects': ['x_data', 'y_data'], 'commit_message': 'data boston_housing added to repository'}


# Repo-Analysis

In [27]:
import repo.plot as plot

In [28]:
for j in range(2):
    training_data = ml_repo._get('training_data')
    training_data.end_index += 50
    ml_repo.add(training_data, message='just modify version number for demonstration purposes')
    for i in range(6,12):
        #print(i)
        param = ml_repo._get('DecisionTreeRegressor/model_param')
        param.sklearn_params['max_depth'] = i
        version = ml_repo.add(param)
        ml_repo.add(param)
        ml_repo.run_training()
        ml_repo.run_evaluation()
        ml_repo.run_measures()
    

In [29]:
#import repo.plot_helper as plt_helper
#if False:
#print(pd.DataFrame(
#plt_helper.get_measure_by_model_parameter(ml_repo, 'DecisionTreeRegressor/measure/test_data/r2', 'max_depth')
#))
plot.measure_by_model_parameter(ml_repo, 'DecisionTreeRegressor/measure/training_data/r2', 'max_depth')


In [30]:
depp = ml_repo._get( 'DecisionTreeRegressor/measure/training_data/r2', version = (0,100))
depp = ml_repo._get( 'DecisionTreeRegressor/measure/training_data/r2', version = (0,100))
job_runner.get_info()

TypeError: get_info() missing 1 required positional argument: 'jobid'

In [ ]:
print(str(depp))